In [ ]:
import pandas as pd

df = pd.read_csv("vendas_certo.csv")

df_grouped = df.groupby("id_da_compra")["produto"].unique().reset_index()
df_grouped.rename(columns={"produto": "produtos"}, inplace=True)

Primeiro eu transformo a lista de produtos agrupada por compra, logo depois eu codifico as transações. Após isso eu gero os conjuntos frequentes com FPGrowth e também gero as regras com base no lift. Depois removo as colunas desnecessárias. Por fim adiciono um ID sequencial e reorganizo para mostrar o ID primeiro.

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
import pandas as pd

transactions = df_grouped["produtos"].tolist()

te = TransactionEncoder()
te_ary = te.fit_transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = fpgrowth(df_encoded, min_support=0.02, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

colunas_desejadas = ['antecedents', 'consequents', 'support', 'confidence', 'lift']
rules_limpo = rules[colunas_desejadas]

rules_limpo = rules_limpo.reset_index(drop=True)
rules_limpo['id'] = rules_limpo.index + 1

rules_limpo = rules_limpo[['id'] + colunas_desejadas]

rules_limpo.head(100)


,id,antecedents,consequents,support,confidence,lift
0,1,(Sabonete),(Pasta de Dente),0.200603,0.716898,2.393476
1,2,(Pasta de Dente),(Sabonete),0.200603,0.669746,2.393476
2,3,"(Sabonete, Queijo Mussarela)",(Pasta de Dente),0.034151,0.604960,2.019754
3,4,"(Pasta de Dente, Queijo Mussarela)",(Sabonete),0.034151,0.556491,1.988737
4,5,(Sabonete),"(Pasta de Dente, Queijo Mussarela)",0.034151,0.122044,1.988737
...,...,...,...,...,...,...
95,96,"(Sabonete, Shampoo)",(Pasta de Dente),0.044784,0.834731,2.786881
96,97,"(Pasta de Dente, Shampoo)",(Sabonete),0.044784,0.658578,2.353568
97,98,(Sabonete),"(Pasta de Dente, Shampoo)",0.044784,0.160045,2.353568
98,99,(Pasta de Dente),"(Sabonete, Shampoo)",0.044784,0.149519,2.786881


Nessa parte eu filtro as regras com lift > 2.0 e confidence > 0.7. Depois mostro as 20 principais regras fortes. Por fim faço uma função para gerar recomendações com base em um produto, nesse caso utilizei a 'Pasta de Dente'.

In [ ]:
regras_fortes = rules_limpo[
    (rules_limpo["lift"] > 2.0) & 
    (rules_limpo["confidence"] > 0.7)
].sort_values(by=["lift", "confidence"], ascending=False)

print("Regras fortes de associação:")
print(regras_fortes.head(20))

def gerar_recomendacoes(produto, regras):
    resultado = regras[
        regras["antecedents"].astype(str).str.contains(produto, case=False)
    ].sort_values(by="lift", ascending=False)
    
    if resultado.empty:
        print(f"Nenhuma recomendação encontrada para '{produto}'.")
    else:
        print(f"Recomendações para '{produto}':")
        for _, row in resultado.iterrows():
            consequente = ', '.join(list(row['consequents']))
            print(f"- {consequente} | confiança: {row['confidence']:.2f} | lift: {row['lift']:.2f}")

gerar_recomendacoes("Pasta de Dente", regras_fortes)


Regras fortes de associação:
      id                       antecedents         consequents   support  \
407  408                          (Carvão)           (Cerveja)  0.024050   
262  263                   (Arroz, Feijão)   (Molho de Tomate)  0.022800   
258  259                           (Arroz)   (Molho de Tomate)  0.062568   
210  211          (Leite Integral, Açúcar)              (Café)  0.032384   
364  365   (Papel Higiênico, Papel Toalha)      (Desinfetante)  0.042367   
362  363   (Desinfetante, Papel Higiênico)      (Papel Toalha)  0.042367   
250  251                    (Café, Feijão)            (Açúcar)  0.036334   
354  355       (Pasta de Dente, Amaciante)          (Sabonete)  0.038084   
173  174          (Manteiga, Pão de Forma)  (Queijo Mussarela)  0.050434   
339  340              (Manteiga, Presunto)  (Queijo Mussarela)  0.048801   
130  131  (Queijo Mussarela, Óleo de Soja)          (Manteiga)  0.048967   
377  378         (Sabonete, Condicionador)    (Pasta de Den

Eu junto todas as listas de produtos em uma só. Logo após eu conto a frequência de cada produto e por fim mostra os 10 produtos mais vendidos.

In [ ]:
from collections import Counter

todos_produtos = [produto for lista in df_grouped["produtos"] for produto in lista]

contagem = Counter(todos_produtos)

mais_vendidos = contagem.most_common(10)

print("📦 Produtos mais vendidos:")
for produto, quantidade in mais_vendidos:
    print(f"- {produto}: {quantidade} vendas")
mais_vendido = contagem.most_common(1)[0]
print(f"🥇 Produto mais vendido: {mais_vendido[0]} ({mais_vendido[1]} vendas)")


📦 Produtos mais vendidos:
- Pasta de Dente: 17971 vendas
- Queijo Mussarela: 17761 vendas
- Manteiga: 17078 vendas
- Sabonete: 16789 vendas
- Açúcar: 15448 vendas
- Café: 15201 vendas
- Papel Toalha: 13149 vendas
- Desinfetante: 13066 vendas
- Condicionador: 10422 vendas
- Molho de Tomate: 10156 vendas
🥇 Produto mais vendido: Pasta de Dente (17971 vendas)
